In [1]:
import requests     # how python goes onto the internet!  Only does static content
import re           # regex
from bs4 import BeautifulSoup # a python HTML parser (version 3)

In [2]:
# let's turn to stock prices
# http://finance.google.com/finance?q=aapl

In [3]:
symbol = 'aapl'

In [4]:
# Let's grab the raw html from the page
r = requests.get('http://finance.google.com/finance?q='+symbol) # the url of the google finance page goes in here

In [5]:
#b = BeautifulSoup(r.text, "html5lib") # create a beautifulsoup object
b = BeautifulSoup(r.text, 'html.parser') # try this line instead if you have problems

In [10]:
b.prettify() # will print the html nicely

u'<!DOCTYPE html>\n<html>\n <head>\n  <script>\n   (function(){(function(){function e(a){this.t={};this.tick=function(a,c,b){var d=void 0!=b?b:(new Date).getTime();this.t[a]=[d,c];if(void 0==b)try{window.console.timeStamp("CSI/"+a)}catch(e){}};this.tick("start",null,a)}var a,d;window.performance&&(d=(a=window.performance.timing)&&a.responseStart);var f=0<d?new e(d):new e;window.jstiming={Timer:e,load:f};if(a){var c=a.navigationStart;0<c&&d>=c&&(window.jstiming.srt=d-c)}if(a){var b=window.jstiming.load;0<c&&d>=c&&(b.tick("_wtsrt",void 0,c),b.tick("wtsrt_","_wtsrt",\nd),b.tick("tbsd_","wtsrt_"))}try{a=null,window.chrome&&window.chrome.csi&&(a=Math.floor(window.chrome.csi().pageT),b&&0<c&&(b.tick("_tbnd",void 0,window.chrome.csi().startE),b.tick("tbnd_","_tbnd",c))),null==a&&window.gtbExternal&&(a=window.gtbExternal.pageT()),null==a&&window.external&&(a=window.external.pageT,b&&0<c&&(b.tick("_tbnd",void 0,window.external.startE),b.tick("tbnd_","_tbnd",c))),a&&(window.jstiming.pt=a)}catch(

In [11]:
# find all span tags
b.findAll('span')
# b.find_all('span') for bs4 users

[<span class="gbi" id="gbn"></span>,
 <span class="gbf" id="gbf"></span>,
 <span id="gbe"></span>,
 <span class="fjfe-searchbox-button-wrapper"><span class="fjfe-searchbox-button-wrapper2"><input class="fjfe-searchbox-button" tabindex="2" type="submit" value="Get quotes"/></span></span>,
 <span class="fjfe-searchbox-button-wrapper2"><input class="fjfe-searchbox-button" tabindex="2" type="submit" value="Get quotes"/></span>,
 <span class="fjfe-recentquotes-duration">(<a href="//www.google.com/history/optout?hl=en">30 days</a>)</span>,
 <span class="fjfe-chg-toggle">chg</span>,
 <span class="fjfe-perc-toggle">%</span>,
 <span class="pr">\n<span id="ref_22144_l">109.22</span>\n</span>,
 <span id="ref_22144_l">109.22</span>,
 <span class="ch bld"><span class="chr" id="ref_22144_c">-0.16</span>\n<span class="chr" id="ref_22144_cp">(-0.15%)</span>\n</span>,
 <span class="chr" id="ref_22144_c">-0.16</span>,
 <span class="chr" id="ref_22144_cp">(-0.15%)</span>,
 <span class="bld" id="ref_22144

In [12]:
# the pattern we recognized from the website
re_tag = re.compile("ref_\d+_l") 

In [13]:
#this tag finds the tag with the price in it!!!!
span_tag = b.find('span', attrs={'id': re_tag}) 
# use find to return THE ONE AND ONLY span tags with an id that matches our regex
# use findAll to find all matches

In [14]:
quote = span_tag.text
quote

u'109.22'

In [27]:
print span_tag

<span id="ref_22144_l">109.22</span>


In [22]:
def get_stock_price(stock):
    r = requests.get('http://finance.google.com/finance?q='+stock)
    b = BeautifulSoup(r.text, 'html.parser')
    re_tag = re.compile("ref_\d+_l")
    span_tag = b.find('span', attrs={'id': re_tag}) 
    if span_tag:
        return span_tag.text
    else:
        return 'no stock found'

In [23]:
print get_stock_price('asdf')

no stock found


In [46]:
##### EXERCISE #####
# Make a method  get_stock_price that takes in ANY stock ticker and grabs the current price
# If the stock ticker doesn't exist, return -1








In [24]:
def get_stock_price(ticker):
    response = requests.get("http://google.com/finance?q="+ticker)
    parser = BeautifulSoup(response.text, "html.parser")
    pattern = re.compile("ref_\d+_l")
    span_tag = parser.find("span", attrs={"id":pattern})
    if span_tag:
        return span_tag.text
    else:
        return -1
    
    
    

In [25]:
for ticker in ['ge', 'spy']:
    print get_stock_price(ticker)

31.29
218.38


In [47]:
###### UFO #######

In [26]:
r = requests.get("http://www.nuforc.org/webreports/ndxe201608.html")
b = BeautifulSoup(r.text, 'html.parser')

In [18]:
# Let's take a look at the first sighting
for tr in b.findAll('tr', attrs = {'valign':'TOP'})[:1]:
    # the findChildren method returns all children underneath it
    for child in tr.findChildren():
        print child.text

8/2/16 00:45
8/2/16 00:45
8/2/16 00:45
Alexandria (UK/Scotland)
Alexandria (UK/Scotland)



Light
Light
2 minutes
2 minutes
Myself and my partner where out smoking when we both seen a light (the same size as a star but brighter) travelling at great speed (abo
Myself and my partner where out smoking when we both seen a light (the same size as a star but brighter) travelling at great speed (abo
8/2/16
8/2/16


In [19]:
# OK, it's a bit messy, Let's clean it up
# Looks like the first element is the date, the 4th is the city, 6th if state, 8th is shape (this ones blank)
# 13th is the summary

In [20]:
ufo_sightings = {
        'Date':[],
        'City':[],
        'State':[],
        'Shape':[],
        'Summary':[]
    }

for tr in b.findAll('tr', attrs = {'valign':'TOP'}):
    # the findChildren method returns all children underneath it
    ufo_sighting_info = []
    for child in tr.findChildren():
        ufo_sighting_info.append(child.text)
    ufo_sightings['Date'].append(ufo_sighting_info[0])
    ufo_sightings['City'].append(ufo_sighting_info[3])
    ufo_sightings['State'].append(ufo_sighting_info[5])
    ufo_sightings['Shape'].append(ufo_sighting_info[7])
    ufo_sightings['Summary'].append(ufo_sighting_info[12])

ufo_sightings

{'City': [u'Alexandria (UK/Scotland)',
  u'Hiltons',
  u'Port Colbourne (Ontario)(Canada)',
  u'Zurich (Switzerland)',
  u'Shoreline',
  u'Shelbyville',
  u'Colorado Springs',
  u'Omaha',
  u'Jacksonville',
  u'Lakewood'],
 'Date': [u'8/2/16 00:45',
  u'8/1/16 22:45',
  u'8/1/16 22:30',
  u'8/1/16 22:00',
  u'8/1/16 21:45',
  u'8/1/16 21:40',
  u'8/1/16 21:30',
  u'8/1/16 21:27',
  u'8/1/16 20:45',
  u'8/1/16 03:25'],
 'Shape': [u'',
  u'Formation',
  u'Light',
  u'',
  u'Unknown',
  u'Light',
  u'Formation',
  u'Circle',
  u'Changing',
  u'Light'],
 'State': [u'', u'VA', u'ON', u'', u'WA', u'IN', u'CO', u'NE', u'FL', u'CO'],
 'Summary': [u'Myself and my partner where out smoking when we both seen a light (the same size as a star but brighter) travelling at great speed (abo',
  u'Two slow moving strobing lights moving steady in a western direction.',
  u'LOTS OF ACTIVITY OVER LAKE ERIE DURING FIREWORKS ON CIVIC HOLIDAY.',
  u'Four floating lights, hovered about 500m above ground then d

In [21]:
import pandas as pd

In [22]:
pd.DataFrame(ufo_sightings) # MAGIC

,City,Date,Shape,State,Summary
0,Alexandria (UK/Scotland),8/2/16 00:45,,,Myself and my partner where out smoking when w...
1,Hiltons,8/1/16 22:45,Formation,VA,Two slow moving strobing lights moving steady ...
2,Port Colbourne (Ontario)(Canada),8/1/16 22:30,Light,ON,LOTS OF ACTIVITY OVER LAKE ERIE DURING FIREWOR...
3,Zurich (Switzerland),8/1/16 22:00,,,"Four floating lights, hovered about 500m above..."
4,Shoreline,8/1/16 21:45,Unknown,WA,"I noticed a light way up in the sky, and quick..."
5,Shelbyville,8/1/16 21:40,Light,IN,Orange orb gliding above tree line from east t...
6,Colorado Springs,8/1/16 21:30,Formation,CO,5 colored lights moving in sky east to west.
7,Omaha,8/1/16 21:27,Circle,NE,Black ufo spotted on a late night drive.
8,Jacksonville,8/1/16 20:45,Changing,FL,"Three bright reds lights crossing the sky, and..."
9,Lakewood,8/1/16 03:25,Light,CO,Sky cracked open.


In [ ]:
# SOME MORE EXAMPLES

In [ ]:
# NY TIMES ARTICLES ON HOME PAGE

In [27]:
response = requests.get("http://www.nytimes.com/")
parser = BeautifulSoup(response.text, "html.parser")
for story in parser.findAll("h2", attrs={'class':'story-heading'}):
    print story.text

Release of Code Raises Fears That the N.S.A. Was Hacked
Early Voting Leaves Trump With Less Time to Catch Up
Ousted Fox News Chief Ailes Advising Trump for Debates
Clinton Warns Supporters to Avoid Complacency 8:42 PM ET
F.B.I. Gives Congress Documents Related to Clinton E-Mail Case 5:38 PM ET
 Podcast: Why They Don’t Trust Her
Death Toll Rises as Louisiana Faces a Flood ‘Disaster’
For a Glimpse of Climate Change, Look to the South 

                                            Deadly Floods                                    
Hand in Hand: Did German Twins’ Finish Cross a Line?

                                            Meet the Biles Family                                    
Rio Today: Simone Biles Soars to Fourth Gold Medal
Your Evening Briefing
18 Recipes to Replace That Sad Desk Lunch
How to Get the Most out of Visiting World-Famous Sites
John McLaughlin, Scrappy Political Pundit, Dies at 89
Univision Is Said to Buy Gawker for $135 Million
Evictions by Armed Men Rattle Mexican T

In [28]:
# WIKIPEDIA FEATURED ARTICLE
response = requests.get("https://en.wikipedia.org/wiki/Wikipedia:Today%27s_featured_article")
parser = BeautifulSoup(response.text, "html.parser")

In [29]:
parser.find('div', attrs={'id':'mw-content-text'}).text

u'\n\n\nToday\'s featured article\n\nAt the top of the Main Page, a summarized lead section from one of Wikipedia\'s featured articles is displayed as "Today\'s featured article" (TFA). The current month\'s queue can be found here. TFAs are scheduled by the TFA coordinators, Brianboulton (Brian), Crisco 1492 (Chris) and Dank (Dan). Community discussion of suggestions takes place at the TFA requests page.\nIf you notice an error in a future TFA summary, you\'re welcome to fix it yourself, but if the mistake is in today\'s or tomorrow\'s summary, you can leave a message at WP:ERRORS to ask an administrator to fix it. The summaries are formatted as a single paragraph of around 1,150 characters (including spaces), with no reference tags or alternative names. Only the link to the specified featured article is bolded, and this must be the first link. The summary should be preceded by an appropriate image when available; fair use images are not allowed.\nThe editnotice template for Today\'s F